In [1]:
# Import Dependencies 
from flask import Flask, jsonify
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import datetime as dt

In [2]:
# Initilize the Flask app
app = Flask(__name__)

In [3]:
# Set up database
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

# Save references to the tables
Measurement = Base.classes.measurement
Station = Base.classes.station

/var/folders/r6/yhgds1l94vl2k1823y88zj4r0000gn/T/ipykernel_84441/3526811208.py:4: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [4]:
# Create API Routes
@app.route("/")
def welcome():
    return (
        f"Welcome to the Hawaii Climate API!<br/><br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )


In [5]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Open session
    session = Session(engine)

    # Find the most recent date in the data
    most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    most_recent_date = dt.datetime.strptime(most_recent_date, '%Y-%m-%d')
    one_year_ago = most_recent_date - dt.timedelta(days=365)

    # Query for the last 12 months of precipitation data
    precipitation_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()

    # Close session
    session.close()

    # Convert query results to dictionary
    precipitation_dict = {date: prcp for date, prcp in precipitation_data}

    return jsonify(precipitation_dict)


In [6]:
@app.route("/api/v1.0/stations")
def stations():
    # Open session
    session = Session(engine)

    # Query all stations
    stations_data = session.query(Station.station).all()

    # Close session
    session.close()

    # Unpack the station data into a list
    stations_list = [station[0] for station in stations_data]

    return jsonify(stations_list)


In [7]:
@app.route("/api/v1.0/tobs")
def tobs():
    # Open session
    session = Session(engine)

    # Find the most active station
    most_active_station = session.query(Measurement.station, func.count(Measurement.station)) \
        .group_by(Measurement.station) \
        .order_by(func.count(Measurement.station).desc()) \
        .first()[0]

    # Find the most recent date and the date one year ago
    most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    most_recent_date = dt.datetime.strptime(most_recent_date, '%Y-%m-%d')
    one_year_ago = most_recent_date - dt.timedelta(days=365)

    # Query the temperature observations for the most active station in the last year
    tobs_data = session.query(Measurement.date, Measurement.tobs) \
        .filter(Measurement.station == most_active_station) \
        .filter(Measurement.date >= one_year_ago) \
        .all()

    # Close session
    session.close()

    # Convert query results to a list of temperature observations
    tobs_list = [tobs[1] for tobs in tobs_data]

    return jsonify(tobs_list)


In [8]:
@app.route("/api/v1.0/<start>")
def temperature_start(start):
    # Open session
    session = Session(engine)

    # Query min, max, and avg temperatures from the start date
    temperature_data = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)) \
        .filter(Measurement.date >= start).all()

    # Close session
    session.close()

    return jsonify({
        "Start Date": start,
        "Min Temperature": temperature_data[0][0],
        "Avg Temperature": temperature_data[0][1],
        "Max Temperature": temperature_data[0][2]
    })


In [9]:
@app.route("/api/v1.0/<start>/<end>")
def temperature_start_end(start, end):
    # Open session
    session = Session(engine)

    # Query min, max, and avg temperatures for the date range
    temperature_data = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)) \
        .filter(Measurement.date >= start) \
        .filter(Measurement.date <= end).all()

    # Close session
    session.close()

    return jsonify({
        "Start Date": start,
        "End Date": end,
        "Min Temperature": temperature_data[0][0],
        "Avg Temperature": temperature_data[0][1],
        "Max Temperature": temperature_data[0][2]
    })
